In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import euclidean_distances
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import math

def haversine(lat1, lon1, lat2, lon2):
    # Convertir latitud y longitud de grados a radianes
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Diferencias entre las coordenadas
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Radio de la Tierra en kilómetros (6371 km)
    km = 6371 * c
    return km
#Para que muestre todas las columnas
pd.set_option('display.max_columns', None)

Primero voy a quitar las columnas que no nos interesan y voy a crear un nuevo dataset únicamente con las columnas que 
nos interesan.

In [3]:

original = pd.read_csv('Bahrein_2021-04-28_1900_2021-05-05_2359-AdaBoost_1_Pre_Matrix.csv')

#Columnas que nos interesan
columnas = ['Latitude', 'Longitude', 'Cell_latitude','Cell_longitude','N1_GCID','N2_GCID','N3_GCID','N4_GCID','N1_RSRP','N2_RSRP','N3_RSRP','N4_RSRP']  
nuevo = original[columnas]
# Lo guardo en un archivo csv
nuevo.to_csv('nuevo_dataset.csv', index = False)  
df = pd.read_csv('nuevo_dataset.csv')

Ahora voy a quitar las muestras que tengan 3 o 4 medidas de potencia y guardarlas en un nuevo dataset que usaremos para la validación.


In [8]:

# Especificar las columnas que deseas comprobar
columnas_comprobar = ['N1_RSRP', 'N2_RSRP', 'N3_RSRP', 'N4_RSRP']  # Reemplaza con los nombres de las columnas que deseas comprobar

# Verificar en qué filas hay al menos 3 valores distintos de 0
filas_a_mover = df[(df[columnas_comprobar] != 0).sum(axis=1) > 2]

# Guardar las filas identificadas en otro dataset
df_validacion = filas_a_mover.copy()

# Eliminar las filas identificadas del dataset original
df = df.drop(filas_a_mover.index)

# Guardar el dataset restante como df_entr
df_entr = df.copy()

# Redondear las columnas de latitud y longitud
columnas_redondeo = ['Latitude', 'Longitude']
df_entr[columnas_redondeo] = df_entr[columnas_redondeo].round(3)

# Eliminar columnas no deseadas y concatenar
columnas_a_eliminar = ['Latitude', 'Longitude', 'Cell_latitude', 'Cell_longitude']
df_cop = df_entr.drop(columnas_a_eliminar, axis=1)
df_entr = pd.concat([df_entr[columnas_redondeo], df_cop], axis=1)

# Guardar los datasets modificados en archivos CSV
df_validacion.to_csv('validacion.csv', index=False)
df_entr.to_csv('entrenamiento.csv', index=False)



Ahora necesito agrupar los valores de entrenamienot cuando la latitud y la longitud son iguales

In [9]:
df_entrenamiento = pd.read_csv('entrenamiento.csv')
df_ordenado = df_entrenamiento.sort_values(by=['Latitude','Longitude'])


Tengo que quitar todos los N y ponerlos en una columna común, primero voy a quitar la columna 3 y 4 porque no tienen información ya que está en los datos de validación.


In [11]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['N3_GCID', 'N4_GCID', 'N3_RSRP', 'N4_RSRP']

# Verificar y eliminar columnas de forma segura, lo he hecho así porque me daba un error raro. Así no da el error, porque una vez se elimina si lo ejecutas otra vez no funciona
for columna in columnas_a_eliminar:
    if columna in df_ordenado.columns:
        df_ordenado = df_ordenado.drop(columna, axis=1)
    else:
        print(f"La columna {columna} no existe en el DataFrame")



Una vez eliminadas lo que haré será juntar estas columnas en una única columna, esto lo haré combinándolas. 

In [12]:
#Primero separo en dos dataframe N1 y N2

df_N1 = df_ordenado[['Latitude', 'Longitude',  'N1_GCID', 'N1_RSRP']].copy()
df_N2 = df_ordenado[['Latitude', 'Longitude',  'N2_GCID', 'N2_RSRP']].copy()

#Ahora las renombro y le pongo a ambas el nombre N 
df_N1.rename(columns={'N1_GCID': 'N_GCID', 'N1_RSRP': 'N_RSRP'}, inplace=True)
df_N2.rename(columns={'N2_GCID': 'N_GCID', 'N2_RSRP': 'N_RSRP'}, inplace=True)

#Ahora combinamos ambos

df_combinado = pd.concat([df_N1, df_N2], axis=0, ignore_index=True)


,Latitude,Longitude,N_GCID,N_RSRP
0,26.200,50.595,179274779,13
1,26.200,50.595,179271975,29
2,26.200,50.595,179281679,17
3,26.200,50.595,179271955,29
4,26.200,50.595,179271975,28
...,...,...,...,...
2898429,26.282,50.626,0,0
2898430,26.282,50.626,0,0
2898431,26.282,50.626,0,0
2898432,26.282,50.626,179356171,30


Ahora tengo que guardar en un nuevo dataset las medias de las potencias en esa misma celda 


In [13]:
df_mean = df_combinado.groupby(['Latitude', 'Longitude', 'N_GCID'])['N_RSRP'].mean().reset_index()
print(df_mean)
df_mean.to_csv('df_mean.csv',index = False)


       Latitude  Longitude     N_GCID     N_RSRP
0        26.200     50.595          0   0.000000
1        26.200     50.595  179271955  34.166667
2        26.200     50.595  179271971  11.000000
3        26.200     50.595  179271975  35.714286
4        26.200     50.595  179274779  13.000000
...         ...        ...        ...        ...
98586    26.282     50.626  179233830  26.000000
98587    26.282     50.626  179240725  28.000000
98588    26.282     50.626  179240729  30.000000
98589    26.282     50.626  179356171  30.000000
98590    26.282     50.626  179356195  30.000000

[98591 rows x 4 columns]


## REESTRUCTURACIÓN DE VALORES 

In [14]:
#dataset modificado
df_comp = df_mean.pivot_table(index=['Latitude', 'Longitude'], columns='N_GCID', values='N_RSRP', fill_value=0)
df_comp.drop(columns= [0], inplace = True)
if 0 in df_comp.columns:
    df_comp = df_comp.drop(columns=[0])
df_comp.to_csv('comp.csv', index = False)
# Ver las primeras filas del DataFrame modificado
primera_fila = df_comp.iloc[[1]]


## REESTRUCTURACIÓN DE VALORES VALIDACIÓN

tengo que hacer lo mismo para los datos de validacion. Primero tenemos que ponerlo de modo que solo tengamos N_GCID y N_RSRP

In [15]:
eliminar = ['Cell_latitude','Cell_longitude']
if all(col in df_validacion.columns for col in eliminar):
    df_validacion = df_validacion.drop(eliminar, axis = 1)


,Latitude,Longitude,N1_GCID,N2_GCID,N3_GCID,N4_GCID,N1_RSRP,N2_RSRP,N3_RSRP,N4_RSRP
13,26.26887,50.60700,179325977,179251983,179324953,179335705,29,28,23,23
20,26.23090,50.59069,179250703,179284259,179250723,0,62,56,55,0
23,26.24611,50.65214,179397387,179388959,179227413,179348501,44,43,43,42
37,26.24154,50.65566,179295509,179251467,179249685,0,67,60,60,0
51,26.24504,50.65895,179251475,179399197,179281171,0,47,46,44,0
...,...,...,...,...,...,...,...,...,...,...
1722958,26.25673,50.60921,179242773,179246603,179261205,0,49,47,45,0
1722976,26.25001,50.65491,179200291,179399203,179399193,179200281,52,51,51,50
1722977,26.24587,50.61118,179201039,179252249,179399439,179397647,42,38,30,29
1722981,26.26353,50.62586,179286539,179356191,179225099,179283211,61,58,55,55


In [16]:

#Primero separo en dos dataframe N1 y N2

df_N1 = df_validacion[['Latitude','Longitude','N1_GCID', 'N1_RSRP']].copy()
df_N2 = df_validacion[['Latitude','Longitude','N2_GCID', 'N2_RSRP']].copy()
df_N3 = df_validacion[['Latitude','Longitude','N3_GCID', 'N3_RSRP']].copy()
df_N4 = df_validacion[['Latitude','Longitude','N4_GCID', 'N4_RSRP']].copy()

#Ahora las renombro y le pongo a ambas el nombre N 
df_N1.rename(columns={'N1_GCID': 'N_GCID', 'N1_RSRP': 'N_RSRP'}, inplace=True)
df_N2.rename(columns={'N2_GCID': 'N_GCID', 'N2_RSRP': 'N_RSRP'}, inplace=True)
df_N3.rename(columns={'N3_GCID': 'N_GCID', 'N3_RSRP': 'N_RSRP'}, inplace=True)
df_N4.rename(columns={'N4_GCID': 'N_GCID', 'N4_RSRP': 'N_RSRP'}, inplace=True)

#Ahora combinamos ambos

df_combinado = pd.concat([df_N1, df_N2, df_N3, df_N4], axis=0, ignore_index=True)
df_combinado = df_combinado.groupby(['Latitude', 'Longitude', 'N_GCID']).mean()



In [17]:
df_val = df_combinado.pivot_table(index=['Latitude', 'Longitude'], columns='N_GCID', values='N_RSRP', fill_value=0)
if 0 in df_val.columns:
    df_val = df_val.drop(columns=[0])



In [2]:
#PONER LOS DOS DATAFRAME DE LA MISMA DIMENSION
# Obtener las columnas que están presentes en df_val pero no en df_comp
columnas_faltantes = df_val.columns.difference(df_comp.columns)

# Obtener las columnas que están presentes en df_val y también en df_comp
columnas_comunes = df_val.columns.intersection(df_comp.columns)

# Filtrar df_comp para que solo contenga las columnas presentes en df_val
df_comp_filtrado = df_comp[columnas_comunes]

# Crear un DataFrame con las columnas faltantes llenas de NaN
df_columnas_faltantes = pd.DataFrame(pd.NA, index=df_comp_filtrado.index, columns=columnas_faltantes)

# Concatenar los DataFrames para agregar las columnas faltantes de una vez
df_comp_filtrado = pd.concat([df_comp_filtrado, df_columnas_faltantes], axis=1)

# Reordenar las columnas de df_comp_filtrado para que coincidan con df_val
df_comp_filtrado = df_comp_filtrado[df_val.columns]

# Verificar las dimensiones de los DataFrames
print(df_comp_filtrado.shape)
print(df_val.shape)


NameError: name 'df_val' is not defined

## COMPARACIÓN

Por último lo que hay que hacer es comparar una fila de validacion para adivinar la latitud y la longitud
Voy a hacerlo por ejemplo para la primera fila la de 26,20-50.59

In [19]:
# Seleccionar una fila de df_val para comparar (por ejemplo, la primera fila)
fila_comparar = 1
fila_val = df_val.iloc[fila_comparar].values.reshape(1, -1)

# Reemplazar NA en los DataFrames con 0
df_val.fillna(0, inplace=True)
df_comp_filtrado.fillna(0, inplace=True)


# Calcular las distancias euclidianas
distances = euclidean_distances(df_comp_filtrado, fila_val)

# Encontrar el índice de la fila más cercana
closest_index = distances.argmin()


C:\Users\Administrador\AppData\Local\Temp\ipykernel_8048\2980195036.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_comp_filtrado.fillna(0, inplace=True)


In [24]:
# Crear una lista para almacenar los resultados
resultados_lista = []

# Reemplazar NA en los DataFrames con 0
df_val.fillna(0, inplace=True)
df_comp_filtrado.fillna(0, inplace=True)

# Iterar sobre cada fila de df_val
for fila_comparar in range(len(df_val)):
    # Imprimir el progreso cada 100 iteraciones
    if fila_comparar % 100 == 0:
        print(f'Procesadas {fila_comparar} filas de {len(df_val)}')

    fila_val = df_val.iloc[fila_comparar].values.reshape(1, -1)
    
    # Calcular las distancias euclidianas
    distances = euclidean_distances(df_comp_filtrado, fila_val)
    
    # Encontrar el índice de la fila más cercana
    closest_index = distances.argmin()
    
    # Extraer las coordenadas originales y predichas desde el índice
    lat_original, lon_original = df_val.index[fila_comparar]
    lat_predicha, lon_predicha = df_comp_filtrado.index[closest_index]
    distancia_minima = distances.min()
    
    # Añadir los resultados a la lista
    resultados_lista.append({
        'lat_original': lat_original,
        'lon_original': lon_original,
        'lat_predicha': lat_predicha,
        'lon_predicha': lon_predicha,
        'distancia_minima': distancia_minima
    })

# Convertir la lista de resultados en un DataFrame
resultados = pd.DataFrame(resultados_lista)

# Mostrar el DataFrame resultante
print(resultados)

Procesadas 0 filas de 181317
Procesadas 100 filas de 181317
Procesadas 200 filas de 181317
Procesadas 300 filas de 181317
Procesadas 400 filas de 181317
Procesadas 500 filas de 181317
Procesadas 600 filas de 181317
Procesadas 700 filas de 181317
Procesadas 800 filas de 181317
Procesadas 900 filas de 181317
Procesadas 1000 filas de 181317
Procesadas 1100 filas de 181317
Procesadas 1200 filas de 181317
Procesadas 1300 filas de 181317
Procesadas 1400 filas de 181317
Procesadas 1500 filas de 181317
Procesadas 1600 filas de 181317
Procesadas 1700 filas de 181317
Procesadas 1800 filas de 181317
Procesadas 1900 filas de 181317
Procesadas 2000 filas de 181317
Procesadas 2100 filas de 181317
Procesadas 2200 filas de 181317
Procesadas 2300 filas de 181317
Procesadas 2400 filas de 181317
Procesadas 2500 filas de 181317
Procesadas 2600 filas de 181317
Procesadas 2700 filas de 181317
Procesadas 2800 filas de 181317
Procesadas 2900 filas de 181317
Procesadas 3000 filas de 181317
Procesadas 3100 fila

In [26]:
#distancia Haversine
resultados['distancia_haversine'] = resultados.apply(
    lambda row: haversine(row['lat_original'], row['lon_original'], row['lat_predicha'], row['lon_predicha']),
    axis=1
)

resultados.to_csv('resultados_3decimales.csv', index = False)  


In [27]:
resultados_2decimales = pd.read_csv("resultados_2decimales.csv")
# Calcular el Error Cuadrático Medio para latitud y longitud
mse_latitud = mean_squared_error(resultados_2decimales['lat_original'], resultados_2decimales['lat_predicha'])
mse_longitud = mean_squared_error(resultados_2decimales['lon_original'], resultados_2decimales['lon_predicha'])

print(f"Error Cuadrático Medio para la latitud con 2 decimales: {mse_latitud}")
print(f"Error Cuadrático Medio para la longitud con 2 decimales: {mse_longitud}")


resultados_3decimales = pd.read_csv("resultados_3decimales.csv")
# Calcular el Error Cuadrático Medio para latitud y longitud
mse_latitud = mean_squared_error(resultados_3decimales['lat_original'], resultados_2decimales['lat_predicha'])
mse_longitud = mean_squared_error(resultados_3decimales['lon_original'], resultados_2decimales['lon_predicha'])

print(f"Error Cuadrático Medio para la latitud con 3 decimales: {mse_latitud}")
print(f"Error Cuadrático Medio para la longitud con 3 decimales: {mse_longitud}")


Error Cuadrático Medio para la latitud con 2 decimales: 0.0012377216421658797
Error Cuadrático Medio para la longitud con 2 decimales: 0.003987702019064047
Error Cuadrático Medio para la latitud con 3 decimales: 0.0012377216421658797
Error Cuadrático Medio para la longitud con 3 decimales: 0.003987702019064047
